In [1]:
import os
import numpy as np

from glob import glob
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize

from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from stardist import Rays_GoldenSpiral
from stardist.models import Config3D, StarDist3D, StarDistData3D

from skimage.segmentation import relabel_sequential

In [2]:
from pathlib import Path

In [3]:
def allocateOnEmptyGPU():
    import os
    import re
    import numpy as np
    from subprocess import check_output

    nvidia_smi_output = check_output(['nvidia-smi']).decode("utf-8")
    memory_matches = re.findall('\d+MiB\s*/\s*\d+MiB', nvidia_smi_output)
    memory_string = [match.split('MiB')[0] for match in memory_matches]
    gpu_memory_usage = list(map(int, memory_string))

    os.environ['CUDA_VISIBLE_DEVICES'] = str(np.argmin(gpu_memory_usage))
    print('Run on GPU with ID: {}'.format(os.environ['CUDA_VISIBLE_DEVICES']))

    return

In [4]:
allocateOnEmptyGPU()

Run on GPU with ID: 0


In [5]:
X_trn_paths = sorted(Path(r'Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\images').glob('*.tif'))
X_vld_paths = sorted(Path(r'Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\valid\images').glob('*.tif'))

for p in X_trn_paths:
    print(p)
    
for p in X_vld_paths:
    print(p)

Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\images\im0.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\images\im1.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\images\im2.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\images\im3.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\images\im4.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\images\im5.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\images\im6.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\valid\images\im0.tif


In [6]:
X_trn = [imread(p) for p in tqdm(X_trn_paths)]
X_vld = [imread(p) for p in tqdm(X_vld_paths)]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.69it/s]


In [7]:
Y_trn_paths = sorted(Path(r'Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\masks').glob('*.tif'))
Y_vld_paths = sorted(Path(r'Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\valid\masks').glob('*.tif'))

In [8]:
for p in Y_trn_paths:
    print(p)
    
for p in Y_vld_paths:
    print(p)

Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\masks\im0.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\masks\im1.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\masks\im2.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\masks\im3.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\masks\im4.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\masks\im5.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\train\masks\im6.tif
Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz100\valid\masks\im0.tif


In [9]:
Y_trn = [imread(p) for p in tqdm(Y_trn_paths)]
Y_vld = [imread(p) for p in tqdm(Y_vld_paths)]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.19it/s]


In [10]:
X_trn[2] = X_trn[2][1:]

In [11]:
for y, x in zip(Y_trn, X_trn):
    print(y.shape, x.shape)

(201, 719, 297) (201, 719, 297)
(90, 421, 491) (90, 421, 491)
(75, 197, 237) (75, 197, 237)
(119, 223, 243) (119, 223, 243)
(72, 647, 541) (72, 647, 541)
(234, 487, 592) (234, 487, 592)
(83, 402, 422) (83, 402, 422)


In [12]:
#X_trn, Y_trn, X_vld, Y_vld = tuple([x[:64, :128, :128] for x in X] for X in [X_trn, Y_trn, X_vld, Y_vld])

In [13]:
#for y, x in zip(Y_trn, X_trn):
#    print(y.shape, x.shape)

In [14]:
modelname = 'eva-v1_dz100_rep1'
basedir = 'models'
n_rays = 192
del_empty_patches = False
percentage = 100
train_patch_size = (64, 192, 192)

In [15]:
    axis_norm = (0, 1, 2)
    X_trn= [normalize(x, 1, 99.8, axis=axis_norm) for x in tqdm(X_trn)]
    X_vld= [normalize(x, 1, 99.8, axis=axis_norm) for x in tqdm(X_vld)]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.26it/s]


In [16]:
    n_channel = 1

    extents = calculate_extents(Y_trn[2])
    anisotropy = tuple(np.max(extents) / extents)

In [17]:
anisotropy

(1.5833333333333333, 1.0, 1.0)

In [18]:
use_gpu = gputools_available()

In [19]:
def random_fliprot(img, mask, axis=None): 
    if axis is None:
        axis = tuple(range(mask.ndim))
    axis = tuple(axis)
            
    assert img.ndim>=mask.ndim
    perm = tuple(np.random.permutation(axis))
    transpose_axis = np.arange(mask.ndim)
    for a, p in zip(axis, perm):
        transpose_axis[a] = p
    transpose_axis = tuple(transpose_axis)
    img = img.transpose(transpose_axis + tuple(range(mask.ndim, img.ndim))) 
    mask = mask.transpose(transpose_axis) 
    for ax in axis: 
        if np.random.rand() > 0.5:
            img = np.flip(img, axis=ax)
            mask = np.flip(mask, axis=ax)
    return img, mask 

def random_intensity_change(img):
    img = img*np.random.uniform(0.6,2) + np.random.uniform(-0.2,0.2)
    return img

def augmenter(x, y):
    """Augmentation of a single input/label image pair.
    x is an input image
    y is the corresponding ground-truth label image
    """
    # Note that we only use fliprots along axis=(1,2), i.e. the yx axis 
    # as 3D microscopy acquisitions are usually not axially symmetric
    x, y = random_fliprot(x, y, axis=(1,2))
    x = random_intensity_change(x)
    return x, y

In [20]:
    # Predict on subsampled grid for increased efficiency and larger field of view
    grid = tuple(1 if a > 1.5 else 2 for a in anisotropy)

    # Use rays on a Fibonacci lattice adjusted for measured anisotropy of the training data
    rays = Rays_GoldenSpiral(n_rays, anisotropy=anisotropy)

    conf = Config3D (
        rays=rays,
        grid=grid,
        anisotropy=anisotropy,
        use_gpu=use_gpu,
        n_channel_in=n_channel,
        # adjust for your data below (make patch size as large as possible)
        train_patch_size=train_patch_size,
        train_batch_size=1,
    )
    vars(conf)

{'n_dim': 3,
 'axes': 'ZYXC',
 'n_channel_in': 1,
 'n_channel_out': 193,
 'train_checkpoint': 'weights_best.h5',
 'train_checkpoint_last': 'weights_last.h5',
 'train_checkpoint_epoch': 'weights_now.h5',
 'n_rays': 192,
 'grid': (1, 2, 2),
 'anisotropy': (1.5833333333333333, 1.0, 1.0),
 'backbone': 'resnet',
 'rays_json': {'name': 'Rays_GoldenSpiral',
  'kwargs': {'n': 192, 'anisotropy': (1.5833333333333333, 1.0, 1.0)}},
 'resnet_n_blocks': 4,
 'resnet_kernel_size': (3, 3, 3),
 'resnet_kernel_init': 'he_normal',
 'resnet_n_filter_base': 32,
 'resnet_n_conv_per_block': 3,
 'resnet_activation': 'relu',
 'resnet_batch_norm': False,
 'net_conv_after_resnet': 128,
 'net_input_shape': (None, None, None, 1),
 'net_mask_shape': (None, None, None, 1),
 'train_patch_size': (64, 192, 192),
 'train_background_reg': 0.0001,
 'train_foreground_only': 0.9,
 'train_dist_loss': 'mae',
 'train_loss_weights': (1, 0.2),
 'train_epochs': 400,
 'train_steps_per_epoch': 100,
 'train_learning_rate': 0.0003,
 '

In [21]:
    if use_gpu:
        from csbdeep.utils.tf import limit_gpu_memory
        # adjust as necessary: limit GPU memory to be used by TensorFlow to leave some to OpenCL-based computations
        limit_gpu_memory(0.8, total_memory=10000)

    model = StarDist3D(conf,
                       name=modelname,
                       basedir=basedir)

    fov = np.array(model._axes_tile_overlap('ZYX'))

    median_size = calculate_extents(Y_trn, np.median)

    if any(median_size > fov):
        print("WARNING: median object size larger than field of view of the neural network.")

    model.train(X_trn, Y_trn,
                validation_data=(X_vld, Y_vld),
                epochs=400,
                augmenter=augmenter)

Using default values: prob_thresh=0.5, nms_thresh=0.4.
Train for 100 steps, validate on 1 samples
Epoch 1/400
100/100 [==============================] - 180s 2s/step - loss: 1.3974 - prob_loss: 0.4498 - dist_loss: 4.7378 - prob_kld: 0.3748 - dist_relevant_mae: 4.7376 - dist_relevant_mse: 37.4066 - val_loss: 1.0447 - val_prob_loss: 0.3396 - val_dist_loss: 3.5253 - val_prob_kld: 0.2915 - val_dist_relevant_mae: 3.5251 - val_dist_relevant_mse: 30.3693
Epoch 2/400
100/100 [==============================] - 174s 2s/step - loss: 1.0183 - prob_loss: 0.2556 - dist_loss: 3.8133 - prob_kld: 0.1845 - dist_relevant_mae: 3.8129 - dist_relevant_mse: 26.4745 - val_loss: 0.8500 - val_prob_loss: 0.0804 - val_dist_loss: 3.8478 - val_prob_kld: 0.0328 - val_dist_relevant_mae: 3.8475 - val_dist_relevant_mse: 27.5016
Epoch 3/400
100/100 [==============================] - 173s 2s/step - loss: 0.7354 - prob_loss: 0.1277 - dist_loss: 3.0384 - prob_kld: 0.0532 - dist_relevant_mae: 3.0379 - dist_relevant_mse: 17.

Epoch 24/400
100/100 [==============================] - 173s 2s/step - loss: 0.3540 - prob_loss: 0.0924 - dist_loss: 1.3081 - prob_kld: 0.0210 - dist_relevant_mae: 1.3078 - dist_relevant_mse: 4.7722 - val_loss: 0.3083 - val_prob_loss: 0.0568 - val_dist_loss: 1.2574 - val_prob_kld: 0.0092 - val_dist_relevant_mae: 1.2570 - val_dist_relevant_mse: 4.5659
Epoch 25/400
100/100 [==============================] - 172s 2s/step - loss: 0.3576 - prob_loss: 0.0958 - dist_loss: 1.3091 - prob_kld: 0.0172 - dist_relevant_mae: 1.3087 - dist_relevant_mse: 4.8782 - val_loss: 0.2939 - val_prob_loss: 0.0566 - val_dist_loss: 1.1865 - val_prob_kld: 0.0091 - val_dist_relevant_mae: 1.1862 - val_dist_relevant_mse: 4.2111
Epoch 26/400
100/100 [==============================] - 172s 2s/step - loss: 0.3388 - prob_loss: 0.0878 - dist_loss: 1.2550 - prob_kld: 0.0154 - dist_relevant_mae: 1.2547 - dist_relevant_mse: 4.6979 - val_loss: 0.2960 - val_prob_loss: 0.0554 - val_dist_loss: 1.2027 - val_prob_kld: 0.0078 - val

100/100 [==============================] - 172s 2s/step - loss: 0.3013 - prob_loss: 0.0889 - dist_loss: 1.0620 - prob_kld: 0.0138 - dist_relevant_mae: 1.0617 - dist_relevant_mse: 3.4248 - val_loss: 0.2682 - val_prob_loss: 0.0545 - val_dist_loss: 1.0684 - val_prob_kld: 0.0069 - val_dist_relevant_mae: 1.0681 - val_dist_relevant_mse: 3.4346
Epoch 48/400
100/100 [==============================] - 172s 2s/step - loss: 0.2940 - prob_loss: 0.0843 - dist_loss: 1.0483 - prob_kld: 0.0144 - dist_relevant_mae: 1.0479 - dist_relevant_mse: 3.3934 - val_loss: 0.2439 - val_prob_loss: 0.0553 - val_dist_loss: 0.9434 - val_prob_kld: 0.0077 - val_dist_relevant_mae: 0.9431 - val_dist_relevant_mse: 2.9393
Epoch 49/400
100/100 [==============================] - 171s 2s/step - loss: 0.2829 - prob_loss: 0.0834 - dist_loss: 0.9975 - prob_kld: 0.0131 - dist_relevant_mae: 0.9972 - dist_relevant_mse: 3.2390 - val_loss: 0.2460 - val_prob_loss: 0.0554 - val_dist_loss: 0.9531 - val_prob_kld: 0.0078 - val_dist_relevan

100/100 [==============================] - 171s 2s/step - loss: 0.2652 - prob_loss: 0.0778 - dist_loss: 0.9369 - prob_kld: 0.0124 - dist_relevant_mae: 0.9366 - dist_relevant_mse: 2.8902 - val_loss: 0.2352 - val_prob_loss: 0.0552 - val_dist_loss: 0.8998 - val_prob_kld: 0.0077 - val_dist_relevant_mae: 0.8995 - val_dist_relevant_mse: 2.4609
Epoch 71/400
100/100 [==============================] - 171s 2s/step - loss: 0.2741 - prob_loss: 0.0832 - dist_loss: 0.9544 - prob_kld: 0.0124 - dist_relevant_mae: 0.9541 - dist_relevant_mse: 2.9012 - val_loss: 0.2292 - val_prob_loss: 0.0545 - val_dist_loss: 0.8730 - val_prob_kld: 0.0070 - val_dist_relevant_mae: 0.8728 - val_dist_relevant_mse: 2.4966
Epoch 72/400
100/100 [==============================] - 171s 2s/step - loss: 0.2729 - prob_loss: 0.0840 - dist_loss: 0.9445 - prob_kld: 0.0131 - dist_relevant_mae: 0.9442 - dist_relevant_mse: 2.8700 - val_loss: 0.2302 - val_prob_loss: 0.0546 - val_dist_loss: 0.8779 - val_prob_kld: 0.0070 - val_dist_relevan

100/100 [==============================] - 172s 2s/step - loss: 0.2632 - prob_loss: 0.0789 - dist_loss: 0.9212 - prob_kld: 0.0113 - dist_relevant_mae: 0.9208 - dist_relevant_mse: 2.6631 - val_loss: 0.2351 - val_prob_loss: 0.0564 - val_dist_loss: 0.8933 - val_prob_kld: 0.0089 - val_dist_relevant_mae: 0.8930 - val_dist_relevant_mse: 2.5319
Epoch 94/400
100/100 [==============================] - 172s 2s/step - loss: 0.2598 - prob_loss: 0.0757 - dist_loss: 0.9203 - prob_kld: 0.0107 - dist_relevant_mae: 0.9200 - dist_relevant_mse: 2.5850 - val_loss: 0.2199 - val_prob_loss: 0.0540 - val_dist_loss: 0.8292 - val_prob_kld: 0.0065 - val_dist_relevant_mae: 0.8289 - val_dist_relevant_mse: 2.2566
Epoch 95/400
100/100 [==============================] - 172s 2s/step - loss: 0.2502 - prob_loss: 0.0818 - dist_loss: 0.8419 - prob_kld: 0.0108 - dist_relevant_mae: 0.8416 - dist_relevant_mse: 2.3751 - val_loss: 0.2212 - val_prob_loss: 0.0538 - val_dist_loss: 0.8368 - val_prob_kld: 0.0063 - val_dist_relevan

...

100/100 [==============================] - 172s 2s/step - loss: 0.2101 - prob_loss: 0.0733 - dist_loss: 0.6837 - prob_kld: 0.0071 - dist_relevant_mae: 0.6835 - dist_relevant_mse: 1.6055 - val_loss: 0.1954 - val_prob_loss: 0.0533 - val_dist_loss: 0.7102 - val_prob_kld: 0.0057 - val_dist_relevant_mae: 0.7100 - val_dist_relevant_mse: 1.6610
Epoch 299/400
100/100 [==============================] - 171s 2s/step - loss: 0.2091 - prob_loss: 0.0791 - dist_loss: 0.6502 - prob_kld: 0.0075 - dist_relevant_mae: 0.6500 - dist_relevant_mse: 1.5553 - val_loss: 0.1927 - val_prob_loss: 0.0533 - val_dist_loss: 0.6973 - val_prob_kld: 0.0057 - val_dist_relevant_mae: 0.6971 - val_dist_relevant_mse: 1.6182
Epoch 300/400
100/100 [==============================] - 171s 2s/step - loss: 0.2127 - prob_loss: 0.0790 - dist_loss: 0.6689 - prob_kld: 0.0074 - dist_relevant_mae: 0.6686 - dist_relevant_mse: 1.6241 - val_loss: 0.1937 - val_prob_loss: 0.0535 - val_dist_loss: 0.7012 - val_prob_kld: 0.0059 - val_dist_relev

100/100 [==============================] - 172s 2s/step - loss: 0.2220 - prob_loss: 0.0823 - dist_loss: 0.6986 - prob_kld: 0.0078 - dist_relevant_mae: 0.6984 - dist_relevant_mse: 1.6999 - val_loss: 0.1947 - val_prob_loss: 0.0535 - val_dist_loss: 0.7061 - val_prob_kld: 0.0059 - val_dist_relevant_mae: 0.7059 - val_dist_relevant_mse: 1.6191
Epoch 322/400
100/100 [==============================] - 172s 2s/step - loss: 0.2118 - prob_loss: 0.0746 - dist_loss: 0.6861 - prob_kld: 0.0071 - dist_relevant_mae: 0.6858 - dist_relevant_mse: 1.6554 - val_loss: 0.1932 - val_prob_loss: 0.0534 - val_dist_loss: 0.6988 - val_prob_kld: 0.0058 - val_dist_relevant_mae: 0.6986 - val_dist_relevant_mse: 1.6157
Epoch 323/400
100/100 [==============================] - 171s 2s/step - loss: 0.2180 - prob_loss: 0.0835 - dist_loss: 0.6724 - prob_kld: 0.0079 - dist_relevant_mae: 0.6721 - dist_relevant_mse: 1.6278 - val_loss: 0.1943 - val_prob_loss: 0.0533 - val_dist_loss: 0.7051 - val_prob_kld: 0.0057 - val_dist_relev

100/100 [==============================] - 172s 2s/step - loss: 0.2108 - prob_loss: 0.0759 - dist_loss: 0.6747 - prob_kld: 0.0072 - dist_relevant_mae: 0.6745 - dist_relevant_mse: 1.5813 - val_loss: 0.1940 - val_prob_loss: 0.0535 - val_dist_loss: 0.7027 - val_prob_kld: 0.0059 - val_dist_relevant_mae: 0.7025 - val_dist_relevant_mse: 1.6209
Epoch 345/400
100/100 [==============================] - 172s 2s/step - loss: 0.2103 - prob_loss: 0.0764 - dist_loss: 0.6694 - prob_kld: 0.0070 - dist_relevant_mae: 0.6692 - dist_relevant_mse: 1.5737 - val_loss: 0.1960 - val_prob_loss: 0.0536 - val_dist_loss: 0.7122 - val_prob_kld: 0.0060 - val_dist_relevant_mae: 0.7120 - val_dist_relevant_mse: 1.6540
Epoch 346/400
100/100 [==============================] - 171s 2s/step - loss: 0.2087 - prob_loss: 0.0735 - dist_loss: 0.6763 - prob_kld: 0.0069 - dist_relevant_mae: 0.6760 - dist_relevant_mse: 1.6027 - val_loss: 0.1954 - val_prob_loss: 0.0535 - val_dist_loss: 0.7097 - val_prob_kld: 0.0059 - val_dist_relev

100/100 [==============================] - 171s 2s/step - loss: 0.2054 - prob_loss: 0.0746 - dist_loss: 0.6542 - prob_kld: 0.0067 - dist_relevant_mae: 0.6540 - dist_relevant_mse: 1.5242 - val_loss: 0.1931 - val_prob_loss: 0.0533 - val_dist_loss: 0.6992 - val_prob_kld: 0.0057 - val_dist_relevant_mae: 0.6990 - val_dist_relevant_mse: 1.6139
Epoch 367/400
100/100 [==============================] - 172s 2s/step - loss: 0.2123 - prob_loss: 0.0790 - dist_loss: 0.6667 - prob_kld: 0.0071 - dist_relevant_mae: 0.6664 - dist_relevant_mse: 1.5571 - val_loss: 0.1956 - val_prob_loss: 0.0535 - val_dist_loss: 0.7108 - val_prob_kld: 0.0059 - val_dist_relevant_mae: 0.7106 - val_dist_relevant_mse: 1.6432
Epoch 368/400
100/100 [==============================] - 171s 2s/step - loss: 0.2064 - prob_loss: 0.0753 - dist_loss: 0.6553 - prob_kld: 0.0070 - dist_relevant_mae: 0.6551 - dist_relevant_mse: 1.5092 - val_loss: 0.1944 - val_prob_loss: 0.0535 - val_dist_loss: 0.7049 - val_prob_kld: 0.0059 - val_dist_relev

 99/100 [============================>.] - ETA: 1s - loss: 0.2110 - prob_loss: 0.0801 - dist_loss: 0.6544 - prob_kld: 0.0073 - dist_relevant_mae: 0.6542 - dist_relevant_mse: 1.5395
Epoch 00389: ReduceLROnPlateau reducing learning rate to 1.8750000890577212e-05.
100/100 [==============================] - 171s 2s/step - loss: 0.2104 - prob_loss: 0.0796 - dist_loss: 0.6540 - prob_kld: 0.0072 - dist_relevant_mae: 0.6538 - dist_relevant_mse: 1.5399 - val_loss: 0.1949 - val_prob_loss: 0.0536 - val_dist_loss: 0.7067 - val_prob_kld: 0.0060 - val_dist_relevant_mae: 0.7064 - val_dist_relevant_mse: 1.6298
Epoch 390/400
100/100 [==============================] - 171s 2s/step - loss: 0.2093 - prob_loss: 0.0776 - dist_loss: 0.6586 - prob_kld: 0.0071 - dist_relevant_mae: 0.6584 - dist_relevant_mse: 1.5315 - val_loss: 0.1935 - val_prob_loss: 0.0536 - val_dist_loss: 0.6997 - val_prob_kld: 0.0060 - val_dist_relevant_mae: 0.6995 - val_dist_relevant_mse: 1.6181
Epoch 391/400
100/100 [=====================

In [22]:
model.optimize_thresholds(X_vld, Y_vld)

NMS threshold = 0.5:  75%|█████████████████████████████████▊           | 15/20 [08:18<02:46, 33.24s/it, 0.730 -> 0.625]


Using optimized values: prob_thresh=0.623021, nms_thresh=0.3.
Saving to 'thresholds.json'.


{'prob': 0.6230211612633794, 'nms': 0.3}